In [1]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import operator
import time
import ast
import math
import subprocess
from collections import defaultdict
from scipy.optimize import curve_fit
from srm.db_manager import connect_to_db, execute_sql
#from Utility.dataProcessing import *
import powerlaw

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
db_name, table = "bidisha", "icct20"
cursor_mysql, conn = connect_to_db("localhost", "root", "root")

In [184]:
def generatehtFromFile(filename):
    f = open(filename)
    hashtaglist = []
    for line in f:
        hashtaglist.append(line.split("\t")[0].strip())
    return hashtaglist

In [185]:
def timestamp(hashtags):
    dicthashtag = defaultdict(list)
    maxts = 0
    mints = sys.maxint
    for hashtag in hashtags:
        hashtag = hashtag.strip()
        hashtagMod1 = '\"%'+hashtag.strip()+',%\"'
        hashtagMod2 = '\"%,'+hashtag.strip()+'%\"'
        hashtagMod3 = '\"'+hashtag.strip()+'\"'
        timestamps = [row[0] for row in execute_sql("Select created_at from %s.%s where\
        (hashTags like %s OR hashTags like %s OR hashTags like %s) \
        order by created_at;", (db_name, table, hashtagMod1, hashtagMod2, hashtagMod3))]
        inttimestamps = [int(time.mktime(x.timetuple())) for x in timestamps]
        tempstr=','.join(str(x) for x in inttimestamps)
        if(inttimestamps[0] < mints):
            mints = inttimestamps[0]
        if(inttimestamps[-1] > maxts):
            maxts = inttimestamps[-1]
        dicthashtag[hashtag] = inttimestamps
    return (dicthashtag, maxts, mints)

In [186]:
def generatechainlist(hashtaglist, klistfile):
    for hashtag in hashtaglist:
        hashtag = hashtag.strip()
        f = open(klistfile+hashtag+".txt",'a')
        hashtagMod1 = '\"%'+hashtag.strip()+',%\"'
        hashtagMod2 = '\"%,'+hashtag.strip()+'%\"'
        hashtagMod3 = '\"'+hashtag.strip()+'\"'
        pidlist = [row[0] for row in execute_sql("Select distinct(parent_id_str) FROM %s.%s where\
        (hashTags like %s OR hashTags like %s OR hashTags like %s) and (parent_id_str is not NULL)\
        ;",(db_name, table, hashtagMod1, hashtagMod2, hashtagMod3))]
        for pid in pidlist:
            tweetchain = [int(time.mktime(row[0].timetuple())) for row in execute_sql("Select created_at \
            FROM %s.%s where (hashTags like %s OR hashTags like %s OR hashTags like %s) and (parent_id_str=%s)\
            order by created_at;",(db_name, table, hashtagMod1, hashtagMod2, hashtagMod3,pid))]
            f.write(pid+"\t["+",".join([str(x) for x in tweetchain])+"]\n")             
        f.close()

In [234]:
def createbucketlist(dictHashtag, mints, maxts, noofbuckets):
    bucketsize = (maxts - mints) / noofbuckets
    bucketlist = defaultdict(list)
    
    for hashtag in dictHashtag.keys():
        hashtag = hashtag.strip()
        timeseries = dictHashtag[hashtag]
        #print hashtag, timeseries[0], timeseries[-1]
        for i in range(0,noofbuckets):
            start = mints + i * bucketsize
            end = mints + (i+1) * bucketsize
            #print start, end
            #if(timeseries[0] > end or timeseries[-1]< start):
            count = 0
            index = start + 0.8 * bucketsize
            for t in timeseries:
                if (t >= start and t <=index) :
                    count += 1        
            bucketlist[i].append((hashtag,count))
    
    for i in range(0,noofbuckets):
        hl = bucketlist[i]
        shl = [row[0] for row in sorted(hl, key=lambda tup: tup[1], reverse=True)][:5]
        bucketlist[i] = shl
    #print shl
    print bucketlist
    return bucketlist

In [224]:
def getKarray(klistfile, hashtag, start, end, train, bincount):
    f = open(klistfile+hashtag+".txt")
    finalcount = 0
    #print hashtag
    minchain =  sys.maxint
    maxchain = 1
    chaindict = defaultdict(int)
    c = 0
    reversetimedict = defaultdict(int)
    for line in f:
        timestamplist = ast.literal_eval(line.split("\t")[1])
        #print timestamplist
        count = 1
        c = c+1
        for ts in timestamplist:
            reversetimedict[ts-start] = c
            #if (ts >= start and ts <= end):
                #continue
            #    print "insde"
            count = count + 1
        if maxchain < count:
            maxchain = count
        if minchain > count:
            minchain = count
        finalcount += count
        chaindict[c] = count
        #print c, count
    karray = []
    #print "train", train
    for ts in train:
        if ts in reversetimedict.keys():
            karray.append(chaindict[reversetimedict[ts]])
        else:
            karray.append(1)
    #print minchain, maxchain
    #print karray
    bins = np.linspace(minchain, maxchain, bincount)
    karray_mod = np.array(np.digitize(karray,bins))
    #print karray_mod
    results = powerlaw.Fit(karray_mod)
    print results.power_law.alpha
    return (karray,results.power_law.alpha)    

In [232]:
def getrelativerank(hashtagList, start, end, dictHashtag, rankfilename):
    continuation = (end - start)/4
    #ranklist = defaultdict(list)
    counthash = defaultdict(int)
    s = start
    f = open(rankfilename, "a")
    for i in range(0,4):
        e = s + continuation
        for hashtag in hashtagList:
            hashtag = hashtag.strip()
            ts = dictHashtag[hashtag]
            count = 0
            for t in ts:
                if (t >= start and t<=e):
                    count = count + 1
            counthash[hashtag] = count
        sorted_h = [hashtagList.index(key) for (key, value) in sorted(counthash.items(),key=operator.itemgetter(1),reverse=True)]
        f.write(str(s - start) + "\t"+ str(e-start)+"\t"+"\t".join([str(x) for x in sorted_h])+"\n")
        s = s + continuation
        #print i
    f.close()
        #ranklist[i] = sorted_h
#return ranklist

In [228]:
def createInput(dictHashtag,mints, maxts, noofbuckets, klistfile, outputfile):
    bucketsize = (maxts - mints) / noofbuckets
    bucketlist = createbucketlist(dictHashtag, mints, maxts, noofbuckets)
    #print bucketlist
    for i in range(0, noofbuckets):
        hashtaglist = bucketlist[i]
        start = mints + i * bucketsize
        end = mints + bucketsize + i * bucketsize
        index = start + 0.8 * bucketsize
        fw = open(outputfile+str(i)+"name.txt", "a")
        fw.write(",".join(hashtaglist))
        fw.close()
        f = open(outputfile+str(i)+".txt", "a")
        #print "start", start, end, hashtaglist
        
        for hashtag in hashtaglist:
            #print "hello"
            hashtag = hashtag.strip()
            #print hashtag
            timeseries = dictHashtag[hashtag]
            #relative = [(x - mints) for x in timeseries]
            train = [x for x in timeseries if (x <=index and x>=start and x <= end)]
            test = [x for x in timeseries if  (x > index and x>=start and x<=end)]
            train = [(x-start) for x in train]
            test = [(x-start) for x in test]
            (karray,alpha) = getKarray(klistfile,hashtag,start, end, train, 10000)
            #print "hello2"
            print "train", train
            #print "test", test
            #karray
            f.write(",".join([str(x) for x in karray])+"\n")
            #arrival
            f.write(",".join([str(x) for x in train])+"\n")
            #omega
            f.write("0.01"+"\n")
            #omega_0
            f.write("0.01"+"\n")
            #alpha
            f.write(str(alpha)+"\n")
            #initial parameters
            f.write("0.01,0.01"+"\n")
            #test set
            f.write(",".join(str(x) for x in test)+"\n")
        f.close()
        #break
        getrelativerank(hashtaglist,start, end, dictHashtag, outfile+"rank"+str(i)+".txt")

In [41]:
#main block
#name of the file with hashtaglist
filename = "/home/bidisha/2017-hashtag-code/Data/hashtags/candidateHashtag.txt"
#directory name for the chain files
klistfile = "/home/bidisha/2017-hashtag-code/Data/hashtags/chain/"
hashtaglist = generatehtFromFile(filename)
print hashtaglist

['ICC_T20', 'IndvNZ', 'IndvsPak', 'PakvBaN', 'T20WORLDCUP', 'T20world', 'WorldT20', 'WorldT20Heroes', 'aus', 'cricket', 'iccwt20', 'india', 'indvpak', 'indvspa', 'pakistan', 'worldcup']


In [64]:
generatechainlist(hashtaglist, klistfile)

In [236]:
klistfile = "/home/bidisha/2017-hashtag-code/Data/hashtags/chain/"
(dictHahstag,maxts,mints) = timestamp(hashtaglist)
outfile = "/home/bidisha/2017-hashtag-code/Data/hashtags/output/"
print mints, maxts
createInput(dictHahstag,mints, maxts, 1 , klistfile, outfile)

1457196848 1458402525
defaultdict(<type 'list'>, {0: ['PakvBaN', 'T20WORLDCUP', 'cricket', 'worldcup', 'ICC_T20']})
1.55379818817
train [116229, 367236, 729500, 846492, 848095, 848124, 859627, 859635, 859841, 860047, 861309, 876014, 876517, 877762, 881558, 882574, 886798, 887506, 887508, 887510, 887615, 888839, 890477, 891056, 891124, 891654, 891759, 891933, 893110, 893699, 894594, 895292, 896114, 896204, 896946, 897480, 898885, 898896, 899435, 900910, 900965, 902451, 902480, 902756, 902770, 902937, 902988, 903010, 903133, 903141, 903161, 903181, 903269, 903333, 903357, 903511, 903883, 903966, 904044, 904109, 904349, 904431, 904447, 904513, 904526, 904556, 904643, 904713, 904907, 904961, 904986, 905160, 905174, 905201, 905210, 905652, 905852, 905973, 906015, 906167, 906288, 906313, 906319, 906326, 906368, 906600, 906670, 906715, 906811, 906821, 906848, 906874, 906897, 906900, 907017, 907027, 907032, 907062, 907112, 907327, 907386, 907423, 907572, 907600, 907736, 907765, 907832, 907854,

Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
